<h1 style="font-family: 'Garamond'; font-size:50px"><center>What are the Top Rated Movies on Netflix?</center></h1>

In 2017, Netflix ditched their 5-star rating system, which was rumoured to be a result of Amy Schumer's stand-up Comedy show receiving extremely negative feedback. Although Netflix has a very successful recommender system in place, it can sometimes be difficult for users to determine which movies are considered 'good' before watching them.

In this notebook I will merge the Netflix and IMDB data to determine what the top rated movies on Netflix are according to IMDB users. I will group the data in different ways including:

* by the gender of users
* by the age group of users
* by genre
* by country

# Imports

In [ ]:
pip install pycountry_convert

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2

# Loading data

In [ ]:
netflix = pd.read_csv('../input/netflix-shows/netflix_titles.csv')
nf_movies = netflix[netflix['type'] == 'Movie']
nf_movies.head()

**Column explanations in Netflix dataset:**
* show_id: unique ID for each movie
* type: 'Movie' or 'TV Show' - in this case we're only using the movies
* title: official movie title
* director: name of director
* cast: list of actors' names in movie
* country: country where the movie was produced
* date_added: date that the movie was added to Netflix
* release_year: year that the movie was originally released
* rating: age rating of movie
* duration: duration of the movie in minutes
* listed_in: relevant genres of the movie
* description: short description of the movie plot

The only columns that I will be using here are title, release_year, listed_in and country as title and release_year are important identifiers, listed_in allows us to find the best movies from different genres and country allows us to find the best movies for each country.

In [ ]:
nf = nf_movies[['title', 'release_year', 'listed_in', 'country']]
nf = nf.rename(columns={'release_year': 'year'})
nf.head()

In [ ]:
imdb_movies = pd.read_csv('../input/imdb-extensive-dataset/IMDb movies.csv')
imdb_movies = imdb_movies[['imdb_title_id', 'title', 'year']]
imdb_ratings = pd.read_csv('../input/imdb-extensive-dataset/IMDb ratings.csv')
imdb = pd.merge(imdb_movies, imdb_ratings, on='imdb_title_id')
imdb.head()

In [ ]:
imdb.info()

There are 51 columns here. I am going to reduce this to 31 columns by choosing:
* title: official movie title
* year: release year of the movie
* weighted_average_vote: weighted mean rating by taking into account the total number of votes
* allgenders_0age_avg_vote: mean rating for all users under 18
* allgenders_0age_votes: number of votes by all users under 18
* allgenders_18age_avg_vote: mean rating for all users aged 18-30
* allgenders_18age_votes: number of votes by all users aged 18-30
* allgenders_30age_avg_vote: mean rating for all users aged 30-45
* allgenders_30age_votes: number of votes by all users aged 30-45
* allgenders_45age_avg_vote: mean rating for all users over 45
* allgenders_45age_votes: number of votes by all users over 45
* males_allages_avg_vote: mean rating for all males
* males_allages_votes: number of votes by all males
* males_0age_avg_vote: mean rating for males under 18
* males_0age_votes: number of votes by males under 18
* males_18age_avg_vote: mean rating for males aged 18-30
* males_18age_votes: number of votes by males aged 18-30
* males_30age_avg_vote: mean rating for males aged 30-45
* males_30age_votes: number of votes by males aged 30-45
* males_45age_avg_vote: mean rating for males over 45
* males_45age_votes: number of votes by males over 45
* females_allages_avg_vote: mean rating for all females
* females_allages_votes: number of votes by all females
* females_0age_avg_vote: mean rating for females under 18
* females_0age_votes: number of votes by females under 18
* females_18age_avg_vote: mean rating for females aged 18-30
* females_18age_votes: number of votes by females aged 18-30
* females_30age_avg_vote: mean rating for females aged 30-45
* females_30age_votes: number of votes by females aged 30-45
* females_45age_avg_vote: mean rating for females over 45
* females_45age_votes: number of votes by females over 45

Some of these columns contain null values as a result of users from those groups not rating specific movies. This is particularly true for young age groups on adult movies, and for movies that are less popular. I will just replace these null values with zeros to make the data more manageable. This shouldn't be an issue in this case as zero simply means that the movie isn't popular in that group, but for other projects this usually isn't the best approach.

The reason the number of votes from each age/gender group needs to be included is because IMDB has not calculated the weighted averages for each group. Therefore these need to be calculated before exploring the data. This is important because we are trying to determine the 'best' movies. Let's imagine comparing a movie with 10 star rating from 3 users with a 9.5 star rating from 10000 users - obviously, the latter should be regarded as 'better' because popularity is an important factor.

In [ ]:
imdb = imdb[['title', 'year', 'weighted_average_vote', 'allgenders_0age_avg_vote', 'allgenders_0age_votes', 'allgenders_18age_avg_vote', 'allgenders_18age_votes',
             'allgenders_30age_avg_vote', 'allgenders_30age_votes', 'allgenders_45age_avg_vote', 'allgenders_45age_votes',
             'males_allages_avg_vote', 'males_allages_votes', 'males_0age_avg_vote', 'males_0age_votes',
             'males_18age_avg_vote', 'males_18age_votes', 'males_30age_avg_vote', 'males_30age_votes', 
             'males_45age_avg_vote', 'males_45age_votes', 'females_allages_avg_vote', 'females_allages_votes',
             'females_0age_avg_vote', 'females_0age_votes', 'females_18age_avg_vote', 'females_18age_votes',
             'females_30age_avg_vote', 'females_30age_votes', 'females_45age_avg_vote', 'females_45age_votes']]
imdb = imdb.fillna(0)

In [ ]:
imdb.info()

# Calculating weighted average ratings

In [ ]:
imdb.describe()

**How does IMDB calculate the weighted average?**

IMDB uses the following equation to calculate the weighted averages:

$$wr = \frac{v}{v+m}R+\frac{m}{v+m}C$$

where

*v = number of votes on the movie*

*m = minimum number of votes to be in the top 50 (currently 1000 votes)*

*R = average/mean rating for the movie*

*C = mean rating across whole dataset (5.9 as seen above)*



Thus, we will calculate these weighted averages and replace the relevant columns.

In [ ]:
def weighted_averages(number, avg, row):
    if row[number] != 0.0:
        wr = ((row[number]/(row[number]+1000))*row[avg]) + ((1000/(row[number]+1000))*5.9)
    else:
        wr = 0
    return wr
    
weighted_allgenders_0age = []
weighted_allgenders_18age = []
weighted_allgenders_30age = []
weighted_allgenders_45age = []
weighted_males_allages = []
weighted_males_0age = []
weighted_males_18age = []
weighted_males_30age = []
weighted_males_45age = []
weighted_females_allages = []
weighted_females_0age = []
weighted_females_18age = []
weighted_females_30age = []
weighted_females_45age = []

for i in range(len(imdb)):
    weighted_allgenders_0age.append(weighted_averages('allgenders_0age_votes', 'allgenders_0age_avg_vote', imdb.iloc[i]))
    weighted_allgenders_18age.append(weighted_averages('allgenders_18age_votes', 'allgenders_18age_avg_vote', imdb.iloc[i]))
    weighted_allgenders_30age.append(weighted_averages('allgenders_30age_votes', 'allgenders_30age_avg_vote', imdb.iloc[i]))
    weighted_allgenders_45age.append(weighted_averages('allgenders_45age_votes', 'allgenders_45age_avg_vote', imdb.iloc[i]))
    weighted_males_allages.append(weighted_averages('males_allages_votes', 'males_allages_avg_vote', imdb.iloc[i]))
    weighted_males_0age.append(weighted_averages('males_0age_votes', 'males_0age_avg_vote', imdb.iloc[i]))
    weighted_males_18age.append(weighted_averages('males_18age_votes', 'males_18age_avg_vote', imdb.iloc[i]))
    weighted_males_30age.append(weighted_averages('males_30age_votes', 'males_30age_avg_vote', imdb.iloc[i]))
    weighted_males_45age.append(weighted_averages('males_45age_votes', 'males_45age_avg_vote', imdb.iloc[i]))
    weighted_females_allages.append(weighted_averages('females_allages_votes', 'females_allages_avg_vote', imdb.iloc[i]))
    weighted_females_0age.append(weighted_averages('females_0age_votes', 'females_0age_avg_vote', imdb.iloc[i]))
    weighted_females_18age.append(weighted_averages('females_18age_votes', 'females_18age_avg_vote', imdb.iloc[i]))
    weighted_females_30age.append(weighted_averages('females_30age_votes', 'females_30age_avg_vote', imdb.iloc[i]))
    weighted_females_45age.append(weighted_averages('females_45age_votes', 'females_45age_avg_vote', imdb.iloc[i]))
    
imdb['weighted_allgenders_0age'] = weighted_allgenders_0age
imdb['weighted_allgenders_18age'] = weighted_allgenders_18age
imdb['weighted_allgenders_30age'] = weighted_allgenders_30age
imdb['weighted_allgenders_45age'] = weighted_allgenders_45age
imdb['weighted_males_allages'] = weighted_males_allages
imdb['weighted_males_0age'] = weighted_males_0age
imdb['weighted_males_18age'] = weighted_males_18age
imdb['weighted_males_30age'] = weighted_males_30age
imdb['weighted_males_45age'] = weighted_males_45age
imdb['weighted_females_allages'] = weighted_females_allages
imdb['weighted_females_0age'] = weighted_females_0age
imdb['weighted_females_18age'] = weighted_females_18age
imdb['weighted_females_30age'] = weighted_females_30age
imdb['weighted_females_45age'] = weighted_females_45age

imdb = imdb[['title', 'year', 'weighted_average_vote', 'weighted_allgenders_0age', 'weighted_allgenders_18age', 'weighted_allgenders_30age', 'weighted_allgenders_45age',
            'weighted_males_allages', 'weighted_males_0age', 'weighted_males_18age', 'weighted_males_30age', 'weighted_males_45age',
            'weighted_females_allages', 'weighted_females_0age', 'weighted_females_18age', 'weighted_females_30age', 'weighted_females_45age']]

In [ ]:
imdb.head()

# Merging datasets

In [ ]:
merged = nf.merge(imdb, how="inner", left_on=['title', 'year'], right_on=['title', 'year'])
merged.head()

In [ ]:
merged.info()

Four of the countries are missing. I will see if these can be determined through the movie title and year; if not, I will just delete these rows.

In [ ]:
merged[merged['country'].isnull()]

By quickly researching, I discovered that *'3 Deewarein'* was made in India, *'Been So Long'* was made in the UK, *'Jis Desh Men Ganga Behti Hai'* was made in India, and *'Tanda Tanya'* was made in Indonesia.

In [ ]:
merged.loc[6, 'country'] = 'India'
merged.loc[74, 'country'] = 'United Kingdom'
merged.loc[260, 'country'] = 'India'
merged.loc[506, 'country'] = 'Indonesia'

# Top Movies by Age and Gender groups

In [ ]:
def top10(column, group):
    titles = []
    scores = []
    top10=merged[column].nlargest(10)
    for i in range(len(top10)):
        index = top10.index[i]
        score = top10.iloc[i]
        scores.append(score)
        title = merged['title'].iloc[index]
        titles.append(title)
        print(i+1, '.', title, ':', score)
    sns.set(rc={'figure.figsize':(12,10)})
    ax = sns.barplot(x=titles, y=scores, data=merged, palette=("crest_d"))
    ticks = ax.set_xticklabels(titles, rotation=90, size = 12)
    ticks = ax.set_yticklabels(ax.get_yticks(), size = 12)
    ax.set(xlabel='Movie Title', ylabel='Mean Rating')
    ax.set_title('Best Rated Movies on Netflix' + group, size= 20)
    return ax

In [ ]:
top10('weighted_average_vote', ' Overall')

<h2 style="font-family: 'Garamond'; font-size:30px"> Overall Top Rated Movies on Netflix - 8.9/10
</h2>

<div class="column">
    <img style="float:left; border:5px solid black; width:45%" align=center src="https://images-na.ssl-images-amazon.com/images/I/71c05lTE03L._AC_SY679_.jpg">
</div>
<div class="column">
    <img style="float:right; border:5px solid black; width:45%" align=center src="https://m.media-amazon.com/images/M/MV5BNDE4OTMxMTctNmRhYy00NWE2LTg3YzItYTk3M2UwOTU5Njg4XkEyXkFqcGdeQXVyNjU0OTQ0OTY@._V1_.jpg">
</div>

In [ ]:
top10('weighted_allgenders_0age', ' by Under 18s - Both Genders')

<h2 style="font-family: 'Garamond'; font-size:30px"> Top Rated Movie on Netflix by Under 18s - 7.8/10
</h2>

<div><img style="float:centre; border:5px solid black; width:50%" src="https://m.media-amazon.com/images/M/MV5BMjAxMzY3NjcxNF5BMl5BanBnXkFtZTcwNTI5OTM0Mw@@._V1_.jpg"></div>

In [ ]:
top10('weighted_allgenders_18age', ' by Users Aged 18-30')

<h2 style="font-family: 'Garamond'; font-size:30px"> Top Rated Movies on Netflix by Users Aged 18-30 - 9/10
</h2>

<div class="column">
    <img style="float:left; border:5px solid black; width:45%" align=center src="https://m.media-amazon.com/images/M/MV5BMjAxMzY3NjcxNF5BMl5BanBnXkFtZTcwNTI5OTM0Mw@@._V1_.jpg">
</div>
<div class="column">
    <img style="float:right; border:5px solid black; width:45%" align=center src="https://m.media-amazon.com/images/M/MV5BNDE4OTMxMTctNmRhYy00NWE2LTg3YzItYTk3M2UwOTU5Njg4XkEyXkFqcGdeQXVyNjU0OTQ0OTY@._V1_.jpg">
</div>

Unfortunately for this analysis, I went through each of the other groups and discovered that the top movie is always one of the three shown above: Pulp Fiction, Schindler's List or Inception.

But what about specific genres?

# Top Movies by Genre

In [ ]:
def genrestop10(genre_string, column, group):
    titles = []
    scores = []
    genre = []
    for i in range(len(merged)):
        row = merged.iloc[i]
        if genre_string in row['listed_in']:
            genre.append(row)
    genre = pd.DataFrame(genre)
    top10=genre[column].nlargest(10)
    for i in range(len(top10)):
        index = top10.index[i]
        score = top10.iloc[i]
        scores.append(score)
        title = merged['title'].iloc[index]
        titles.append(title)
        print(i+1, '.', title, ':', score)
    sns.set(rc={'figure.figsize':(12,10)})
    ax = sns.barplot(x=titles, y=scores, palette=("crest_d"))
    ticks = ax.set_xticklabels(titles, rotation=90, size = 12)
    ticks = ax.set_yticklabels(ax.get_yticks(), size = 12)
    ax.set(xlabel='Movie Title', ylabel='Mean Rating')
    ax.set_title('Best Rated ' + genre_string + ' on Netflix' + group, size= 20)
    return ax

<h2 style="font-family: 'Garamond'; font-size:30px"> What is the best Horror movie on Netflix?
</h2>

In [ ]:
genrestop10('Horror', 'weighted_average_vote', ' Overall')

<h2 style="font-family: 'Garamond'; font-size:30px"> Overall Top Rated Horror Movie on Netflix - 8.3/10
</h2>

<div>
    <img style="float:centre; border:5px solid black; width:45%" align=center src="https://occ-0-1722-1723.1.nflxso.net/dnm/api/v6/evlCitJPPCVCry0BZlEFb5-QjKc/AAAABXmVaJkbDwA3aTHEkCh8hHFFgzZnX9Cx4zA1r9KyadqIaNkhuF7iKAEdu_Pi1GMyH8ujLlRMS4-6iQ9n2EbkNrwrgue_.jpg">
</div>

Although this film is widely considered to fall under the Horror genre, it is also mainly classed as Crime/Drama, so perhaps it isn't fair to compare it to other Horrors. If we were to ignore this, Shutter would be the top rated Horror movie with a rating of 7.1/10.

<h2 style="font-family: 'Garamond'; font-size:30px"> What are the best Comedy movies on Netflix?
</h2>

In [ ]:
genrestop10('Comedies', 'weighted_average_vote', ' Overall')

<h2 style="font-family: 'Garamond'; font-size:30px"> Overall Top Rated Comedy Movie on Netflix - 8.6/10
</h2>

<div>
    <img style="float:centre; border:5px solid black; width:45%" align=center src="https://m.media-amazon.com/images/M/MV5BMjA4OTczODgxNF5BMl5BanBnXkFtZTgwMDAzMTU2NDE@._V1_.jpg">
</div>

<h2 style="font-family: 'Garamond'; font-size:30px"> What are the best Family movies on Netflix?
</h2>

In [ ]:
genrestop10('Children & Family Movies', 'weighted_average_vote', ' Overall')

<h2 style="font-family: 'Garamond'; font-size:30px"> Overall Top Rated Family Movie on Netflix - 8/10
</h2>

<div>
    <img style="float:centre; border:5px solid black; width:45%" align=center src="https://www.filmibeat.com/ph-big/2011/09/1317104828496224.jpg">
</div>

<h2 style="font-family: 'Garamond'; font-size:30px"> What are the best Romance movies on Netflix?
</h2>

In [ ]:
genrestop10('Romantic Movies', 'weighted_average_vote', ' Overall')

<h2 style="font-family: 'Garamond'; font-size:30px"> Overall Top Rated Romance Movie on Netflix - 8.6/10
</h2>

<div>
    <img style="float:centre; border:5px solid black; width:45%" align=center src="https://i2.cinestaan.com/image-bank/1500-1500/56001-57000/56690.jpg">
</div>

<h2 style="font-family: 'Garamond'; font-size:30px"> What are the best Action movies on Netflix?
</h2>

In [ ]:
genrestop10('Action', 'weighted_average_vote', ' Overall')

<h2 style="font-family: 'Garamond'; font-size:30px"> Overall Top Rated Action Movie on Netflix - 8.6/10
</h2>

<div>
    <img style="float:centre; border:5px solid black; width:45%" align=center src="https://m.media-amazon.com/images/M/MV5BMjAxMzY3NjcxNF5BMl5BanBnXkFtZTcwNTI5OTM0Mw@@._V1_.jpg">
</div>

<h2 style="font-family: 'Garamond'; font-size:30px"> Distribution of Ratings for Different Genres
</h2>

In [ ]:
subframe = []
first_genre_list = []
for i in range(len(merged)):
    row = merged.iloc[i]
    if 'Horror' in row['listed_in']:
        first_genre_list.append('Horror')
        subframe.append(row)
    elif 'Comedies' in row['listed_in']:
        first_genre_list.append('Comedy')
        subframe.append(row)
    elif 'Children & Family Movies' in row['listed_in']:
        first_genre_list.append('Family')
        subframe.append(row)
    elif 'Romantic Movies' in row['listed_in']:
        first_genre_list.append('Romance')
        subframe.append(row)
    elif 'Action' in row['listed_in']:
        first_genre_list.append('Action')
        subframe.append(row)

subframe = pd.DataFrame(subframe)
subframe['first_genre'] = first_genre_list

sns.displot(subframe, x="weighted_average_vote", hue="first_genre")

While Comedy is definitely the most popular out of these genres, the top rated genre overall out of these five seems to be Action.

# Top Movies by Continent

As we know which country each movie was produced in, we can use this data to group the movies by continent and determine what the top rated movie is for each continent.

An issue with this is that some movies were produced in multiple countries and continents. The country column needs to be changed somehow to allow us to do this. Therefore, I am only going to consider the first country in each list - I am making the assumption that the first country is the main country it was produced in.

In [ ]:
for i in range(len(merged['country'])):
    split = merged['country'].iloc[i].split(',')
    merged['country'].iloc[i] = split[0]
    if merged['country'].iloc[i] == 'Soviet Union':
        merged['country'].iloc[i] = 'Russia'

continents = {
    'AF': 'Africa',
    'AS': 'Asia',
    'OC': 'Australia',
    'EU': 'Europe',
    'NA': 'North America',
    'SA': 'South America'
}
countries = merged['country']

merged['continent'] = [continents[country_alpha2_to_continent_code(country_name_to_country_alpha2(country))] for country in countries]

Lets visualise the proportions of movies from each continent.

In [ ]:
data = merged['continent'].value_counts(normalize=True) * 100
labels = data.keys()

pie, ax = plt.subplots(figsize=[12,12])
patches, texts, autotexts = plt.pie(x=data, autopct="%.1f%%", pctdistance=0.9, labels=labels, textprops={'fontsize': 16}, shadow=True, explode=[0,0.1,0,0,0,0], colors=['#5cb68b','#0886ad','#7cbbdf','#48b0df','#5c936a','#5c9ac1'])
plt.title("Movies from each Continent", fontsize=20);

It's no surprise that 50% of the movies on Netflix are from Asia, as many of the top movies for each genre were Asian.

In [ ]:
def continentstop10(continent_string, column, group):
    titles = []
    scores = []
    continent = []
    for i in range(len(merged)):
        row = merged.iloc[i]
        if row['continent'] == continent_string:
            continent.append(row)
    continent = pd.DataFrame(continent)
    top10=continent[column].nlargest(10)
    for i in range(len(top10)):
        index = top10.index[i]
        score = top10.iloc[i]
        scores.append(score)
        title = merged['title'].iloc[index]
        titles.append(title)
        print(i+1, '.', title, ':', score)
    sns.set(rc={'figure.figsize':(12,10)})
    ax = sns.barplot(x=titles, y=scores, palette=("crest_d"))
    ticks = ax.set_xticklabels(titles, rotation=90, size = 12)
    ticks = ax.set_yticklabels(ax.get_yticks(), size = 12)
    ax.set(xlabel='Movie Title', ylabel='Mean Rating')
    ax.set_title('Best Rated Movies from ' + continent_string + ' on Netflix' + group, size= 20)
    return ax

<h2 style="font-family: 'Garamond'; font-size:30px"> What are the best Asian movies on Netflix?
</h2>

In [ ]:
continentstop10('Asia', 'weighted_average_vote', ' Overall')

<h2 style="font-family: 'Garamond'; font-size:30px"> Overall Top Rated Asian Movies on Netflix - 8.6/10
</h2>

<div class="column">
    <img style="float:left; border:5px solid black; width:33%" align=center src="https://m.media-amazon.com/images/M/MV5BMjA4OTczODgxNF5BMl5BanBnXkFtZTgwMDAzMTU2NDE@._V1_.jpg">
</div>
<div class="column">
    <img style="float:left; border:5px solid black; width:33%" align=center src="https://i2.cinestaan.com/image-bank/1500-1500/56001-57000/56690.jpg">
</div>
<div class="column">
    <img style="float:left; border:5px solid black; width:33%" align=center src="https://upload.wikimedia.org/wikipedia/en/0/08/Oththa_Seruppu_Size_7.jpg">
</div>

<h2 style="font-family: 'Garamond'; font-size:30px"> What are the best African movies on Netflix?
</h2>

In [ ]:
continentstop10('Africa', 'weighted_average_vote', ' Overall')

<h2 style="font-family: 'Garamond'; font-size:30px"> Overall Top Rated African Movie on Netflix - 8.3/10
</h2>

<div class="column">
    <img style="float:centre; border:5px solid black; width:45%" align=center src="https://occ-0-1722-1723.1.nflxso.net/dnm/api/v6/evlCitJPPCVCry0BZlEFb5-QjKc/AAAABXmVaJkbDwA3aTHEkCh8hHFFgzZnX9Cx4zA1r9KyadqIaNkhuF7iKAEdu_Pi1GMyH8ujLlRMS4-6iQ9n2EbkNrwrgue_.jpg">
</div>

<h2 style="font-family: 'Garamond'; font-size:30px"> What are the best Australian movies on Netflix?
</h2>

In [ ]:
continentstop10('Australia', 'weighted_average_vote', ' Overall')

<h2 style="font-family: 'Garamond'; font-size:30px"> Overall Top Rated Australian Movie on Netflix - 7.1/10
</h2>

<div class="column">
    <img style="float:centre; border:5px solid black; width:45%" align=center src="https://m.media-amazon.com/images/M/MV5BYWNkOTc5MjgtMzM4Zi00MjVjLWFjNWItNmRiMjIzZjgzYTM2XkEyXkFqcGdeQXVyMTMxMTY0OTQ@._V1_.jpg">
</div>

<h2 style="font-family: 'Garamond'; font-size:30px"> What are the best European movies on Netflix?
</h2>

In [ ]:
continentstop10('Europe', 'weighted_average_vote', ' Overall')

<h2 style="font-family: 'Garamond'; font-size:30px"> Overall Top Rated European Movies on Netflix - 8.1/10
</h2>

<div class="column">
    <img style="float:left; border:5px solid black; width:45%" align=center src="https://images-na.ssl-images-amazon.com/images/I/718gUcln-8L._AC_SY450_.jpg">
</div>
<div class="column">
    <img style="float:right; border:5px solid black; width:45%" align=center src="https://i.pinimg.com/originals/88/66/65/886665e09c688e0bbb1fd7bc51c57671.jpg">
</div>

<h2 style="font-family: 'Garamond'; font-size:30px"> What are the best North American movies on Netflix?
</h2>

In [ ]:
continentstop10('North America', 'weighted_average_vote', ' Overall')

<h2 style="font-family: 'Garamond'; font-size:30px"> Overall Top Rated North American Movies on Netflix - 8.9/10
</h2>

<div class="column">
    <img style="float:left; border:5px solid black; width:45%" align=center src="https://images-na.ssl-images-amazon.com/images/I/71c05lTE03L._AC_SY679_.jpg">
</div>
<div class="column">
    <img style="float:right; border:5px solid black; width:45%" align=center src="https://m.media-amazon.com/images/M/MV5BNDE4OTMxMTctNmRhYy00NWE2LTg3YzItYTk3M2UwOTU5Njg4XkEyXkFqcGdeQXVyNjU0OTQ0OTY@._V1_.jpg">
</div>

<h2 style="font-family: 'Garamond'; font-size:30px"> What are the best South American movies on Netflix?
</h2>

In [ ]:
continentstop10('South America', 'weighted_average_vote', ' Overall')

<h2 style="font-family: 'Garamond'; font-size:30px"> Overall Top Rated South American Movies on Netflix - 8.6/10
</h2>

<div class="column">
    <img style="float:left; border:5px solid black; width:45%" align=center src="https://m.media-amazon.com/images/M/MV5BOTMwYjc5ZmItYTFjZC00ZGQ3LTlkNTMtMjZiNTZlMWQzNzI5XkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_.jpg">
</div>

# Conclusion

We've seen a nice range of different movies from different groups in this notebook, with Pulp Fiction and Schindler's List coming out on top overall. Of course there is so much more exploring that could be done - exploring more genres, specific countries etc. but this is where I'll end this notebook.

Thank you so much for reading. Hope you enjoyed! 